# Nettoyage des données EXIOBASE3 pour le secteur de l'élevage bovin.

Nettoyage des données d'EXIOBASE3 pour obtenir les DataFrames correspondant aux quantités de terres utilisées dans la production de biens provenant du secteur bovin (comme le pâturage des bovins) et destinées à la consommation finale des régions réceptrices.

#### Vocabulaire:

**Regions sources:** les régions qui utilisent une quantité de terres pour produire des biens destinés à la consommation finale, que ce soit pour leur propre consommation interne ou pour l'exportation vers d'autres régions.
Les cartes thermiques permettent de visualiser le niveau d'exploitation intensive des régions sources pour répondre aux besoins de production.

**Régions réceptrices:** les régions qui importent des biens qui ont nécessité l'utilisation de terres dans leur production. 
Les cartes thermiques permettent de visualiser les régions réceptrices qui dépendent des importations pour satisfaire leur propre consommation finale en biens nécessitant une utilisation intensive des terres.

**Importations de terres :** Les régions réceptrices importent des terres virtuellement à travers les biens qu'elles importent pour satisfaire leur propre consommation finale. Cela signifie que ces régions utilisent des ressources terrestres situées à l'extérieur de leurs frontières pour produire des biens consommés localement.

**Exportations de terres :** Les régions sources exportent des terres virtuelles en produisant des biens destinés à la consommation finale dans d'autres régions. Cela implique que ces régions utilisent leurs ressources terrestres internes pour produire des biens exportés vers d'autres régions.


In [133]:
#importations
import pymrio
import pandas as pd

In [137]:
#Fonction load qui prend 'year' comme argument
def load(year):
    #Chargement des donnée EXIOBASE3 pour chaque année
    path = f"C:/Master 1/Stage/EXIOBASE3/IOT_{year}_ixi.zip"
    exio3 = pymrio.parse_exiobase3(path=path)

    #Diagonalisation de l'impact associé à l'utilisation des terres
    diag_GHG = exio3.impacts.diag_stressor('Land use Crop, Forest, Pasture')

    #Les valeurs de la diagonalisation sont attribués à 'exio3.GHG_source'
    exio3.GHG_source = diag_GHG

    #Commande pymrio qui permet de compléter les parties manquantes de EXIOBASE3
    exio3.calc_all()

    #Conserve uniquement le secteur bovin tous les pays de la DataFrame
    #Regroupe pour les colonnes tous les secteurs pour toutes les régions en additionnant leurs valeurs
    df = exio3.GHG_source.D_cba.loc[(slice(None), 'Cattle farming'), :].groupby(level='region', axis=1).sum()

    #Pour les lignes ne garder que le noms des régions afin d'enlever le Multindex
    new_index = [x[0] if isinstance(x, tuple) else x for x in df.index]
    df.index = new_index

    #Formation de liste pour former de nouvelles régions
    UE = ['AT', 'BE', 'BG', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GR', 'HR', 'HU', 'IE', 'IT', 
        'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']
    MO = ['WM', 'CY']
    Europe = ['WE', 'CH', 'TR', 'GB']
    Asie_Pacifique = ['WA', 'TW', 'KR']
    Afrique = ['WF', 'ZA']

    #Création de nouvelles colonnes pour chaque nouvelle région
    df['UE'] = df[UE].sum(axis=1)
    df['Moyen-Orient'] = df[MO].sum(axis=1)
    df['Europe'] = df[Europe].sum(axis=1)
    df['Asie Pacifique'] = df[Asie_Pacifique].sum(axis=1)
    df['Afrique'] = df[Afrique].sum(axis=1)

    #Création de nouvelles lignes pour chaque nouvelle région
    df.loc['UE'] = df.loc[UE].sum()
    df.loc['Moyen-Orient'] = df.loc[MO].sum()
    df.loc['Europe'] = df.loc[Europe].sum()
    df.loc['Asie Pacifique'] = df.loc[Asie_Pacifique].sum()
    df.loc['Afrique'] = df.loc[Afrique].sum()

    #Supprime les lignes et colonnes des pays individuels inclus dans les nouvelles régions créées.
    df = df.drop(UE + MO + Europe + Asie_Pacifique + Afrique, axis=0)
    df = df.drop(UE + MO + Europe + Asie_Pacifique + Afrique, axis=1)

    #Réorganise l'ordre des pays de la DataFrame pour les lignes et les colonnes
    order = ["Afrique", "CN", "US", "Asie Pacifique", "WL", "BR", "RU", "MX", "AU","Moyen-Orient",  "UE",  "Europe", "CA", "IN", "ID", "JP"]
    df = pd.DataFrame(df, index=order)
    df = df[order]

    #Renomme le noms des lignes et des colonnes pour rendre la DataFrame plus compréhensible
    region = ["Afrique", "Chine","Etats-Unis", "Asie Pacifique",  "Amérique Latine et Caraïbes", "Brésil", "Russie", "Mexique", "Australie", "Moyen-Orient", "Union européenne",  "Europe", "Canada", "Inde", "Indonésie", "Japon" ]
    df.columns = region
    df.index = region

    #Calculer la somme de chaque ligne de la DataFrame pour cette année 
    df_imp = df.sum(axis=0)
    
    #Division de chaque éléments des lignes par la somme obtenue précedemment et mettre en pourcentage
    df1_norm = df.div(df_imp, axis=1) * 100

    #Calculer la somme de chaque colonne de la DataFrame pour cette année 
    df_exp = df.sum(axis=1)
    
    #Division de chaque éléments des colonnes par la somme obtenue précedemment et mettre en pourcentage
    df2_norm = df.div(df_exp, axis=0) * 100

    #Résult de la fonction load
    return df, df1_norm, df2_norm


#### Étapes du nettoyage : <br>
1. Choisir le stresseur ou l'impact qui nous intéresse pour notre analyse. Ici nous utilisons l'impact lié à l'utilisation des terres (agricoles, forêts et pâturages).
2. Diagonalisation de ce même stresseur/impact. Dans ce cas, cela permet d'exprimer la quantité des terres utilisées de chaque secteur et chaque région dans une matrice diagonale.
3. On multiplie la matrice diagonale avec la matrice de consommation finale 'D_cba'. On obtient ainsi une matrice qui détaille pour chaque région source (les régions situées en lignes), la quantité de terres utilisées pour la production de chaque secteur, ainsi que les quantités de terres utilisées par les régions réceptrices (les régions situées en colonnes) pour répondre à leur consommation finale.
4. On nettoie cette nouvelle matrice en ne conservant que les quantités de terres utilisées par les régions sources pour la production du secteur bovin. De plus, on agrège tous les secteurs des régions réceptrices.
5. Réduire le nombre de régions pour diminuer la taille de la DataFrame afin de la rendre plus visible lors de la représentation visuelle. Pour cela, nous gardons certaines régions telles qu'elles sont définies dans EXIOBASE3, comme la Chine, les États-Unis, l'Inde, la Russie, l'Indonésie, le Japon, le Mexique, le Brésil et le Canada. Cependant, nous regroupons :
   - Tous les pays de l'Union européenne dans la région "Union européenne".
   - Tous les pays européens qui ne font pas partie de l'Union européenne dans la région 'Europe'.
   - Tous les pays du Moyen-Orient (hors la Turquie qui est déjà comprise dans la région "Europe") dans la région 'Moyen-Orient'.
   - Tous les pays d'Asie-Pacifique (hors le Japon, la Chine, l'Inde et l'Indonésie) dans la région 'Asie-Pacifique'.
   - Tous les pays d'Afrique (hors l'Égypte qui est déjà comptabilisée dans la région 'Moyen-Orient') dans la région 'Afrique'.
   
   Cela permet de réduire le nombre de régions dans notre DataFrame de 49 à 16.
   
6. On normalise la matrice en sommant chaque ligne, puis on divise chaque valeur de la ligne par cette somme. Cela permet d'obtenir la part des quantités de terres internes des régions sources utilisées pour leur propre consommation finale ou pour les exportations vers les autres régions réceptrices afin de répondre à leur consommation finale.
7. On normalise la matrice en sommant chaque colonne, puis on divise chaque valeur de la colonne par cette somme. Cela permet de visualiser la part des terres utilisées qui sont locales ou externes par les régions réceptrices pour répondre à leur consommation finale, et donc de voir la part des importations pour chaque région réceptrice.

La liste détaillée des pays regroupés dans les 49 régions des données EXIOBASE3 peut être consultée à ce [lien](https://github.com/MarDrd/exiobase3-manipulation/blob/main/R%C3%A9gions.ipynb).

La liste détaillée des pays regroupés dans les 16 nouvelles régions peut être consultée à ce [lien](https://github.com/MarDrd/exiobase3-manipulation/blob/main/Impact%20-%20Consommation%20Finale/R%C3%A9gion%20Description/Composition_Regions.ipynb).


In [146]:
#Chargements des nouvelles bases de données
df, df1_norm, df2_norm = load(2022)

c:\Users\Marie\anaconda3\Lib\site-packages\pymrio\core\mriosystem.py:900: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
c:\Users\Marie\anaconda3\Lib\site-packages\pymrio\core\mriosystem.py:956: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
c:\Users\Marie\anaconda3\Lib\site-packages\pymrio\core\mriosystem.py:900: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
c:\Users\Marie\anaconda3\Lib\site-packages\pymrio\core\mriosystem.py:956: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
c:\Use

In [147]:
#Enregistre chaque base de données en format CSV
df.to_csv('df_CattleFarm_2022.csv')
df1_norm.to_csv('df_CattleFarm_Imp_2022.csv')
df2_norm.to_csv('df_CattleFarm_Exp_2022.csv')